 Q3.Build an RNN to generate text character-by-character using PyTorch.
 - Convert text into one-hot vectors or embeddings.
 - Handle sequential dependencies with hidden states.
 - Train with Teacher Forcing vs. Free Running modes.
 - Implement sampling strategies like greedy search and temperature-based sampling

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import torch.nn.functional as F

In [27]:
text = "Generative AI transforms the way we create and innovate"

In [28]:
unique_chars = sorted(list(set(text)))
char_to_index = {char: i for i, char in enumerate(unique_chars)}
index_to_char = {i: char for char, i in char_to_index.items()}
vocab_size = len(unique_chars)

In [29]:
def text_to_tensor(text):
    return torch.tensor([char_to_index[char] for char in text], dtype=torch.long)

In [30]:
text_tensor = text_to_tensor(text)

In [31]:
embedding_size = 16
hidden_size = 128
num_layers = 2

In [32]:
def init_weights():
    embedding_weights = torch.randn(vocab_size, embedding_size, requires_grad=True)
    rnn_weights = torch.randn(embedding_size, hidden_size, requires_grad=True)
    rnn_hidden_weights = torch.randn(hidden_size, hidden_size, requires_grad=True)
    output_weights = torch.randn(hidden_size, vocab_size, requires_grad=True)
    return embedding_weights, rnn_weights, rnn_hidden_weights, output_weights

embedding_weights, rnn_weights, rnn_hidden_weights, output_weights = init_weights()

In [33]:
def forward_pass(input_char, hidden):
    embed = embedding_weights[input_char]
    hidden = torch.tanh(torch.matmul(embed, rnn_weights) + torch.matmul(hidden.detach(), rnn_hidden_weights))
    output = torch.matmul(hidden, output_weights)
    return output, hidden

In [34]:
def train(text_tensor, epochs=100, learning_rate=0.01, teacher_forcing_ratio=0.5):
    optimizer = optim.Adam([embedding_weights, rnn_weights, rnn_hidden_weights, output_weights], lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        hidden = torch.zeros(hidden_size)
        total_loss = 0
        
        for i in range(len(text_tensor) - 1):
            input_char = text_tensor[i]
            target_char = text_tensor[i + 1].unsqueeze(0)
            
            output, hidden = forward_pass(input_char, hidden)
            
            loss = loss_function(output.unsqueeze(0), target_char)
            total_loss += loss.item()  # Use .item() to detach from computation graph
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Loss = {total_loss:.4f}")  # Avoid calling .item() on total_loss directly


In [35]:
train(text_tensor, epochs=200)

Epoch 0: Loss = 1122.4318
Epoch 10: Loss = 319.1014
Epoch 20: Loss = 148.3770
Epoch 30: Loss = 75.3114
Epoch 40: Loss = 22.6483
Epoch 50: Loss = 38.0736
Epoch 60: Loss = 69.5783
Epoch 70: Loss = 13.5266
Epoch 80: Loss = 44.8423
Epoch 90: Loss = 35.3693
Epoch 100: Loss = 35.3084
Epoch 110: Loss = 7.0400
Epoch 120: Loss = 2.7832
Epoch 130: Loss = 69.4346
Epoch 140: Loss = 19.8190
Epoch 150: Loss = 14.5419
Epoch 160: Loss = 16.5938
Epoch 170: Loss = 35.7077
Epoch 180: Loss = 44.1293
Epoch 190: Loss = 0.1838


In [36]:
def generate_text(start_text, length=100, temperature=1.0):
    hidden = torch.zeros(hidden_size)
    input_sequence = text_to_tensor(start_text)
    output_text = start_text
    
    for _ in range(length):
        output, hidden = forward_pass(input_sequence[-1], hidden)
        output = output / temperature
        probabilities = F.softmax(output, dim=-1).detach().numpy()
        next_char_index = np.random.choice(range(vocab_size), p=probabilities.ravel())
        output_text += index_to_char[next_char_index]
        input_sequence = torch.cat((input_sequence, torch.tensor([next_char_index])), dim=0)
    
    return output_text

In [37]:
print(generate_text("Generative", length=100, temperature=0.8))

Generative Aate AI te create AI nnd innovatewe Aate cre way we create AI AI rate cre way we cre way way we cre
